In [ ]:
%load_ext autoreload
%autoreload 2

import pprint, json, math, os, sys
import fitz
import pandas as pd
from collections import defaultdict

# dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
# fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"

dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'
sys.path.append(os.path.abspath(dir_path))

from app.fundData import *
from app.helper import Helper

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
rep_path = r'\data\output\pdf_report.xlsx'

json_path = dir_path + r'\data\output\dump.json'
pkl_path = dir_path + r'\data\output\sample.pkl'

mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
""" SAMCO PDF FILE MAIN CODE"""

object = Samco(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund['Samco Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)
title = df.title.to_dict()
pages =  [3, 5, 7, 9, 11, 13, 15, 17, 18]

data = object.extract_clipped_data(file_path,pages, title)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
""" TATA FILE MAIN CODE """
object = Tata(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund["Tata Mutual Fund"]

path, df = object.check_and_highlight(file_path, 10)

title = df.title.to_dict()
pages = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]

data = object.extract_clipped_data(file_path,pages, title)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = Helper.drop_dict(extracted_text)
final_data = object.refine_extracted_data(final_text)
Helper.quick_json_dump(final_data, json_path)

In [ ]:
""" FRANKLIN TEMPLETON FILE MAIN CODE"""

object = FranklinTempleton(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund["Franklin Templeton Mutual Fund"]

# path, df = object.check_and_highlight(file_path, 6)
title = df.title.to_dict()
pages = [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

data = object.extract_clipped_data(file_path,pages, title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
# final_dict = Helper.drop_empty_dict_values()
Helper.quick_json_dump(final_text, json_path)


In [ ]:
"""BANDHAN MF FILE MAIN CODE"""

object = Bandhan(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund["Bandhan Mutual Fund"]

path, df = object.check_and_highlight(file_path,6)
pages = [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
title = df.title.to_dict()

data = object.extract_clipped_data(file_path,pages, title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""HELIOS MF FILE MAIN CODE""" #clean using regex

object = Helios(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund['Helios Mutual Fund']

path, df = object.check_and_highlight(file_path,6)
title = df.title.to_dict()
pages =  [2, 4, 6, 8, 10]

data = object.extract_data_relative_line(file_path,pages ,'left', title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)


In [24]:
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)


 JSON saved at C:\Users\rando\OneDrive\Documents\mywork-repo\data\output\dump_helios_21_58.json


In [27]:
for fund, content in final_text.items():
    if "portfolio" in content:
        for text in content['portfolio']:
            print(text)

Scheme Category: Flexi Cap Fund
Benchmark: NIFTY 500 Total Return Index(TRI)
Plans and Options: Direct and Regular Plan, each with
Growth and IDCW Option (Re-investment, Transfer,
Pay-out Sub-Option)
Inception Date : 13th November 2023
Minimum Investment Amount: Rs. 5,000/- (plus in multiple
of Re.1)
Additional Investment Amount: Rs.1,000/- (plus in multiple
of Re.1)
Fund Manager: Mr. Alok Bahl (Since Inception / Overall 32 years
of experience)
Mr. Pratik Singh (Since April 20, 2024 / Overall 10 years of
experience)
Entry Load: NA
Exit Load: (i) If units redeemed or switched out are upto 10%
(limit) of the units purchased or switched in within 3 months
from the date of allotment  Nil
(ii) If units redeemed or switched out are over and above
the limit within 3 months from the date of allotment  1% of
the applicable NAV
(iii) If redeemed/switched out after 3 months from the date
of allotment  Nil
Face Value per Unit: Rs. 10
Scheme Category: Sectoral Fund
Benchmark: NIFTY Financial Servic

In [ ]:
"""EDELWEISS MP FILE MAIN CODE""" #not cleaned data

object = Edelweiss(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund['Edelweiss Mutual Fund']


path, df = object.check_and_highlight(file_path, 6)

# pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]

# title,objectives = object.get_proper_fund_names(file_path,pages)

# data = object.extract_data_relative_line(file_path,pages ,'right',title ) #used line here
# data = object.extract_span_data(data,[])
# clean_data = object.process_text_data(data) 
# nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
# extracted_text = object.get_generated_content(nested_data)

In [8]:
""""HSBC MAIN FILE CODE""" #Issue: Some pages have 2 or more fund data on same page
#some data is present on right side as well so define two lines

object = HSBC(dir_path,dry_path,fin_path)
file_path = mutual_fund['HSBC Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
data = object.extract_data_relative_line(file_path,pages,line_x ,'right', fund_titles) #used line here
data = object.extract_span_data(data,[])

clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
""" INVESCO MF MAIN FILE CODE""" #Issue: two fund on same page

object = Invesco(dir_path,dry_path,fin_path)
file_path = mutual_fund['Invesco Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(4,44)]
data = object.get_clipped_data(file_path,pages,bbox, fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [27]:
""" ITI MAIN FILE CODE""" #Issues: NO so far

object = ITI(dir_path,dry_path,fin_path)
file_path = mutual_fund['ITI Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(9,27)]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""JM FUND MAIN FILE CODE""" #objective on left, sometimes date is with header

object = JMMF(dir_path,dry_path,fin_path)
file_path = mutual_fund['JM Financial Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']

path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")
pages = [22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
"""DSP MAIN FILE CODE""" #Issue: highlight count must be lower
#Issue: data on both side ends so issue coming

object = DSP(dir_path,dry_path,fin_path)
file_path = mutual_fund['DSP Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [ i for i in range(1,66)]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
"""KOTAK FUND MAIN CODE""" #Issue: 1 or more fund on same page, color on headers
object = Kotak(dir_path,dry_path,fin_path)
file_path = mutual_fund['Kotak Mahindra Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 36, 37, 38, 40, 41, 43, 44, 46, 48, 49, 50, 52, 54, 55, 56, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
data = object.extract_data_relative_line(file_path,pages,line_x,'left',fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
#ISSUE: THE FUND NAMES ARE IMAGES """ LIC MF MAIN CODE"""
 #Issue: objective on right of line

object = LIC(dir_path,dry_path,fin_path)
file_path = mutual_fund['Kotak Mahindra Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)


In [24]:
""" MAHINDRA MANULIFE MAIN CODE""" #incrase pages count

object = MahindraManu(dir_path,dry_path,fin_path)
file_path = mutual_fund['Mahindra Manulife Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")


fund_titles = object.get_proper_fund_names(file_path,[i for i in range(1,80)])
pages = [i for i in range(8,34)]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [ ]:
""" MOTILAL OSWAL MAIN CODE FILE"""

object = MotilalOswal(dir_path,dry_path,fin_path)
file_path = mutual_fund['Motilal Oswal Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [i for i in range(4,21)]
data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""NJMF MAIN CODE FILE"""

object = NJMF(dir_path,dry_path,fin_path)
file_path = mutual_fund['NJ Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [1, 5, 7, 9]

data = object.get_clipped_data(file_path,pages,bbox,fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""QUANT MAIN CODE FILE"""

object = QuantMF(dir_path,dry_path,fin_path)
file_path = mutual_fund['Quant Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 61, 63, 65]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
pages = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 61, 63, 65]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""SUNDARAM MAIN FILE CODE"""

object = Sundaram(dir_path,dry_path,fin_path)
file_path = mutual_fund['Sundaram Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""TAURUS MAIN FILE CODE"""

object = Taurus(dir_path,dry_path,fin_path)
file_path = mutual_fund['Taurus Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [13, 14, 15, 16, 17, 19, 20]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""TRUST MAIN FILE CODE""" #Issue: Invest Obj in right #Names coming as headers fund managers

object = Trust(dir_path,dry_path,fin_path)
file_path = mutual_fund['Trust Mutual Fund']

fund_data = object.PARAMS['fund']
line_x = object.PARAMS['line_x']
bbox = object.PARAMS['clip_box']
data_cond = object.PARAMS['data']
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)
msgs = input("\nEnter Y/N: ")

pages = [6, 7, 8, 9, 10, 11, 12, 13]
data = object.extract_data_relative_line(file_path,pages,line_x,"left",fund_titles) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [ ]:
"""UTI MAIN FILE CODE"""

object = UTI(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['UTI Mutual Fund']

# path,df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 41, 43, 45, 46, 48, 50, 52, 54, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77]
data = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)

In [205]:
final_text = object.refine_extracted_data(extracted_text)
# final_dict = Helper.drop_dict(final_text)
Helper.quick_json_dump(final_text, json_path)


 JSON saved at C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\dump_uti_17_58.json


In [ ]:
"""WHITEOAK MAIN FILE CODE""" #Issue: Inv Obj in the right #json issue
object = WhiteOak(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['WhiteOak Mutual Fund']

# path, df = object.check_and_highlight(file_path, 6)

title = df.title.to_dict()
pages = [7, 9, 11, 13, 15, 17, 19, 20, 21, 23, 25, 26, 28, 30, 32, 34, 35]
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_dict = Helper.drop_dict(final_text)
Helper.quick_json_dump(final_dict, json_path)

In [ ]:
'''' 360 ONE MAIN FILE CODE''' #No issues as of now

object = ThreeSixtyOne(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['360 ONE Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [5, 6, 7, 8, 9, 10, 11, 12]
data = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_dict = Helper.drop_dict(final_text)
Helper.quick_json_dump(final_dict, json_path)

In [ ]:
""" BAJAJ FINSERV MAIN FILE CODE""" #major issue of lines #minor of dropping keys of final dict

object = BajajFinServ(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Bajaj finserv Mutual Fund']

#path, df = object.check_and_highlight(file_path, 7)

pages = [15, 17, 19, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33]
title = df.title.to_dict()

#left
data_l = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data_l = object.extract_span_data(data_l,[])
clean_data = object.process_text_data(data_l)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

#right
data_r = object.extract_data_relative_line(file_path,pages,"right",title) #used line here
data_r = object.extract_span_data(data_r,[])
clean_data = object.process_text_data(data_r)
nested_data_right, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

nested_data = {key:{**nested_data_left[key],**nested_data_right[key] }for key in nested_data_left.keys()}
extracted_text = object.get_generated_content(nested_data)


In [ ]:
final_text = object.refine_extracted_data(extracted_text)
final_dict = Helper.drop_dict(final_text)
Helper.quick_json_dump(final_dict, json_path)

In [ ]:
""" BARODA BNP MAIN FILE CODE""" #Issue: Header funds #scheme details

object = BarodaBNP(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Baroda BNP Paribas Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)


pages = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
title = object.get_proper_fund_names(file_path, pages, (0,0,210,75))

data = object.extract_data_relative_line(file_path,pages,"left", title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_dict = Helper.drop_dict(final_text)
Helper.quick_json_dump(final_dict, json_path)


In [ ]:
"""NAVI MAIN FILE CODE""" #Issue: Left and right

object = NAVI(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Navi Mutual Fund']


# path,df = object.check_and_highlight(file_path,5)
title = df.title.to_dict()
pages = [2, 6, 8, 10, 13, 16, 18, 20, 22]

#left
data_l = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data_l = object.extract_span_data(data_l,[])
clean_data = object.process_text_data(data_l)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 9.0)

#right
data_r = object.extract_data_relative_line(file_path,pages,"right",title) #used line here
data_r = object.extract_span_data(data_r,[])
clean_data = object.process_text_data(data_r)
nested_data_right, matrix = object.create_nested_dict(clean_data,30.0, 9.0)

nested_data = {key:{**nested_data_left[key],**nested_data_right[key] }for key in nested_data_left.keys()}
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""ZERODHA MAIN CODE""" 

object = Zerodha(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Zerodha Mutual Fund']

path,df= object.check_and_highlight(file_path,7)
title = df.title.to_dict()
pages = [3, 5, 7, 9, 11, 13, 15]

#left
data_l = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data_l = object.extract_span_data(data_l,[])
clean_data = object.process_text_data(data_l)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

#right
data_r = object.extract_data_relative_line(file_path,pages,"right",title) #used line here
data_r = object.extract_span_data(data_r,[])
clean_data = object.process_text_data(data_r)
nested_data_right, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

nested_data = {key:{**nested_data_left[key],**nested_data_right[key] }for key in nested_data_left.keys()}
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)


In [203]:
pattern =r'^(Regular|Direct.*Option$)\s+([\d]+\.\d+)'
for fund, content in final_text.items():
    if "nav" in content.keys():
        for text in content['nav']:
            print(text)

Regular Growth Option 261.2650
Regular IDCW Option 52.8969
Direct Growth Option 285.1391
Direct IDCW Option 62.0088
Regular Growth Option 304.0538
Regular IDCW Option 143.8810
Direct Growth Option 337.1335
Direct IDCW Option 165.9044
Regular Growth Option 26.7611
Regular IDCW Option 26.7609
Direct Growth Option 28.5895
Direct IDCW Option 28.5895
Regular Growth Option 175.1630
Regular IDCW Option 85.3315
Direct Growth Option 186.7000
Direct IDCW Option 91.7460
Regular Growth Option 163.2167
Regular IDCW Option 47.5657
Direct Growth Option 177.9757
Direct IDCW Option 58.0443
Regular Growth Option 201.4377
Regular IDCW Option 38.4828
Direct Growth Option 224.2575
Direct IDCW Option 52.8493
Regular Growth Option 14.8359
Regular IDCW Option 14.8359
Direct Growth Option 15.6403
Direct IDCW Option 15.6399
Regular Growth Option 317.7719
Regular IDCW Option 210.0906
Direct Growth Option 339.6348
Direct IDCW Option 226.0215
Regular Growth Option 12.4528
Regular IDCW Option 12.4528
Direct Growth 

In [166]:
Helper.quick_json_dump(extracted_text,json_path)       


 JSON saved at C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\dump_whiteoak_15_47.json
